# Regular Expressions, Text Normalization, Edit Distance

**text normalization** - converting text to a more convenient, standard form

**lemmatization** - the task of determining that two words have the same root, despite their surface differences
- sang, sung, and sings are forms of the verb sing
- lemmatizer (a function) maps these words to their lemma, sing

**sentence segmentation** - breaking up a text into individual sentences, using cues like periods or exclamation points

**edit distance** - metric that measures how similar two strings are based on the number of edits (insertions, deletions, substitutions) it takes to change one string into the other

### regular expressions

a language for specifying text search strings

**quick regex review:**
 
disjunction
- `[wW]oodchuck` - Woodchuck or woodchuck
- `[abc]`  - ‘a’, ‘b’, or ‘c’
- `gupp(y|ies)` - guppy or guppies

range and `^` as *negation*
- `[0-9]`  - a single digit 0-9
- `[ ˆA-Z]` - not an upper case letter

optional elements: `?`
- `colou?r` - color or colour

kleene star - zero or more occurrences of the immediately previous character or regular expression
- `[ab]*` - aaaa, ababab, bbbb

wildcard `.`
- `beg.n`: begin, beg’n, begun

anchors
- `ˆThe box\.$` - a line that contains only the phrase `The box`
- /\bthe\b/ - `the` (but not the word other)

### words

**corpus** - a computer-readable collection of text or speech

**utterance** - the spoken correlate of a sentence

*I do uh main- mainly business data processing*

- disfluencies occur in spoken sentences
    - uh and um are called fillers
    - sometimes these helpful because they may signal the restart of a clause or idea

**lemma** - is a set of lexical forms having the same stem, the same major part-of-speech, and the same word sense
- box, boxes


### Text Normalization

1. Tokenizing (segmenting) words
2. Normalizing word formats
3. Segmenting sentences

unix example of tokenizing a text file

`tr -sc ’A-Za-z’ ’\n’ < sh.txt | tr A-Z a-z | sort | uniq -c | sort -n -r`

- changes every sequence of nonalphabetic characters to a newline
- \-c option complements to non-alphabet
- \-s option squeezes all sequences into a single character
- \-n option sorts numerically rather than alphabetically
- \-r option means to sort in reverse order

result:
    
    27378 the
    26084 and
    22538 i
    19771 to
    17481 of
    14725 a
    13826 you
    12489 my
    11318 that
    11112 in
    

**function words** - articles, pronouns, prepositions, the most frequent corpora

**named entity detection** - the task of detecting names, dates, and organizations 


**morpheme** - the smallest meaning-bearing unit of a language

## tokenization
What's the point of tokenization?

- it’s helpful to have subword tokens to deal with unknown words
- ML systems learn facts about words in a training corpus and then use that to make decisions about a test corpus
-  if our training corpus contains, say the words low, and lowest, but not lower, but then the word "lower" appears in our test corpus, our system will be able to combine tokens from the training corpus to understand "lower"
- most tokens are words, but some tokens are frequent morphemes or other subwords like -er, so that an unseen word can be represented by combining the parts


### byte-pair encoding for tokenization

based on a method for text compression, the intuition of the algorithm is to iteratively merge frequent pairs of characters

algorithm:

>1. Initialize vocabulary with the set of symbols equal to all characters seen plus a "_"
>2. Represent each word in the corpus as a combination of the characters along with the special end of word token </w>.
>3. Iteratively count character pairs in all tokens of the vocabulary.
>4. Merge every occurrence of the most frequent pair, add the new character n-gram to the vocabulary.
>5. Repeat step 3 and 4 until the desired number of merge operations are completed or the desired vocabulary size is achieved (which is a hyperparameter).



#### my byte-pair encoding algorithm for tokenization
- inputs: dictionary of words with frequencies of those words
- K: number of iterations to run the algorithm

In [83]:
corpus = '''
Smoke the first 48 hours, grind 22 and sleep two hours
Put 24s on the new Audi, white on white like baby powder
Drop ya' boo off at Fulton County
Might count it up and then re-count it
Double cups like Tunechi, yeah
Bust it down with these goonies, yeah
Give no effs yeah, we don't give no effs, yeah
Go fill my cup yeah, yo go fill my cup, yeah
You heard that the slums made me, I'm cool with the Konvicts
The coupe look like Akon, eff all that bum shit
'''

In [82]:
import re
from collections import Counter, defaultdict

In [98]:
def build_vocab(corpus):
    '''
    Initialize vocabulary, add special char </w> to each word
    ''' 
    chars = [" ".join(word) + " </w>" for word in corpus.split()]
    
    # Count frequency of chars in corpus
    vocab = Counter(chars)

    return vocab

In [117]:
vocab = build_vocab(corpus)
print(vocab)

Counter({'t h e </w>': 4, 'y e a h </w>': 4, 'l i k e </w>': 3, 'i t </w>': 3, 'a n d </w>': 2, 'o n </w>': 2, 'w h i t e </w>': 2, 'w i t h </w>': 2, 'n o </w>': 2, 'y e a h , </w>': 2, 'f i l l </w>': 2, 'm y </w>': 2, 't h a t </w>': 2, 'S m o k e </w>': 1, 'f i r s t </w>': 1, '4 8 </w>': 1, 'h o u r s , </w>': 1, 'g r i n d </w>': 1, '2 2 </w>': 1, 's l e e p </w>': 1, 't w o </w>': 1, 'h o u r s </w>': 1, 'P u t </w>': 1, '2 4 s </w>': 1, 'n e w </w>': 1, 'A u d i , </w>': 1, 'b a b y </w>': 1, 'p o w d e r </w>': 1, 'D r o p </w>': 1, "y a ' </w>": 1, 'b o o </w>': 1, 'o f f </w>': 1, 'a t </w>': 1, 'F u l t o n </w>': 1, 'C o u n t y </w>': 1, 'M i g h t </w>': 1, 'c o u n t </w>': 1, 'u p </w>': 1, 't h e n </w>': 1, 'r e - c o u n t </w>': 1, 'D o u b l e </w>': 1, 'c u p s </w>': 1, 'T u n e c h i , </w>': 1, 'B u s t </w>': 1, 'd o w n </w>': 1, 't h e s e </w>': 1, 'g o o n i e s , </w>': 1, 'G i v e </w>': 1, 'e f f s </w>': 1, 'w e </w>': 1, "d o n ' t </w>": 1, 'g i v e

In [100]:
def get_pair_counts(vocab):
    '''
    Get counts of the pairs of all consecutive symbols
    '''

    pair_counts = defaultdict(int)
    for word, frequency in vocab.items():
        chars = word.split()

        for i in range(len(chars) - 1):
            pair_counts[chars[i], chars[i + 1]] += frequency

    return pair_counts

In [118]:
pair_counts = get_pair_counts(vocab)
print(pair_counts)

defaultdict(<class 'int'>, {('S', 'm'): 1, ('m', 'o'): 1, ('o', 'k'): 2, ('k', 'e'): 4, ('e', '</w>'): 18, ('t', 'h'): 10, ('h', 'e'): 8, ('f', 'i'): 3, ('i', 'r'): 1, ('r', 's'): 3, ('s', 't'): 2, ('t', '</w>'): 14, ('4', '8'): 1, ('8', '</w>'): 1, ('h', 'o'): 2, ('o', 'u'): 8, ('u', 'r'): 2, ('s', ','): 3, (',', '</w>'): 10, ('g', 'r'): 1, ('r', 'i'): 1, ('i', 'n'): 1, ('n', 'd'): 3, ('d', '</w>'): 4, ('2', '2'): 1, ('2', '</w>'): 1, ('a', 'n'): 2, ('s', 'l'): 2, ('l', 'e'): 2, ('e', 'e'): 1, ('e', 'p'): 1, ('p', '</w>'): 4, ('t', 'w'): 1, ('w', 'o'): 1, ('o', '</w>'): 7, ('s', '</w>'): 6, ('P', 'u'): 1, ('u', 't'): 1, ('2', '4'): 1, ('4', 's'): 1, ('o', 'n'): 7, ('n', '</w>'): 5, ('n', 'e'): 2, ('e', 'w'): 1, ('w', '</w>'): 1, ('A', 'u'): 1, ('u', 'd'): 1, ('d', 'i'): 1, ('i', ','): 2, ('w', 'h'): 2, ('h', 'i'): 4, ('i', 't'): 8, ('t', 'e'): 2, ('l', 'i'): 3, ('i', 'k'): 3, ('b', 'a'): 1, ('a', 'b'): 1, ('b', 'y'): 1, ('y', '</w>'): 4, ('p', 'o'): 1, ('o', 'w'): 2, ('w', 'd'): 1, ('

In [119]:
most_frequent = max(pair_counts, key=pair_counts.get)
print(most_frequent)

('e', '</w>')


In [107]:
def merge_most_frequent(pair: tuple, v_in: dict) -> dict:
    '''
    Merge all occurrences of the most frequent pair
    '''
    
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in v_in:
        # replace most frequent pair in all vocabulary
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]

    return v_out

In [120]:
vocab = merge_most_frequent(most_frequent, vocab)
print(vocab)
# you can see the 'e</w>'s being merged in the following vocabulary

{'S m o k e</w>': 1, 't h e</w>': 4, 'f i r s t </w>': 1, '4 8 </w>': 1, 'h o u r s , </w>': 1, 'g r i n d </w>': 1, '2 2 </w>': 1, 'a n d </w>': 2, 's l e e p </w>': 1, 't w o </w>': 1, 'h o u r s </w>': 1, 'P u t </w>': 1, '2 4 s </w>': 1, 'o n </w>': 2, 'n e w </w>': 1, 'A u d i , </w>': 1, 'w h i t e</w>': 2, 'l i k e</w>': 3, 'b a b y </w>': 1, 'p o w d e r </w>': 1, 'D r o p </w>': 1, "y a ' </w>": 1, 'b o o </w>': 1, 'o f f </w>': 1, 'a t </w>': 1, 'F u l t o n </w>': 1, 'C o u n t y </w>': 1, 'M i g h t </w>': 1, 'c o u n t </w>': 1, 'i t </w>': 3, 'u p </w>': 1, 't h e n </w>': 1, 'r e - c o u n t </w>': 1, 'D o u b l e</w>': 1, 'c u p s </w>': 1, 'T u n e c h i , </w>': 1, 'y e a h </w>': 4, 'B u s t </w>': 1, 'd o w n </w>': 1, 'w i t h </w>': 2, 't h e s e</w>': 1, 'g o o n i e s , </w>': 1, 'G i v e</w>': 1, 'n o </w>': 2, 'e f f s </w>': 1, 'y e a h , </w>': 2, 'w e</w>': 1, "d o n ' t </w>": 1, 'g i v e</w>': 1, 'e f f s , </w>': 1, 'G o </w>': 1, 'f i l l </w>': 2, 'm y

In [122]:
# now run this K more times
num_merges = 50
for i in range(num_merges):
    pair_counts = get_pair_counts(vocab)

    if not pair_counts:
        break

    most_frequent = max(pair_counts, key=pair_counts.get)
    vocab = merge_most_frequent(most_frequent, vocab)

    
print(vocab)

{'Smoke</w>': 1, 'the</w>': 4, 'first</w>': 1, '48</w>': 1, 'hours,</w>': 1, 'grind</w>': 1, '22</w>': 1, 'and</w>': 2, 'sleep</w>': 1, 'two</w>': 1, 'hours</w>': 1, 'Put</w>': 1, '24s</w>': 1, 'on</w>': 2, 'new</w>': 1, 'Audi,</w>': 1, 'white</w>': 2, 'like</w>': 3, 'baby</w>': 1, 'powder</w>': 1, 'D r o p</w>': 1, "y a ' </w>": 1, 'b o o</w>': 1, 'o ff </w>': 1, 'at</w>': 1, 'F u l t on</w>': 1, 'C oun t y</w>': 1, 'M i g h t</w>': 1, 'count</w>': 1, 'it</w>': 3, 'u p</w>': 1, 'the n</w>': 1, 'r e - count</w>': 1, 'D ou b l e</w>': 1, 'cup s</w>': 1, 'T u ne c hi ,</w>': 1, 'yeah</w>': 4, 'B u st</w>': 1, 'd ow n</w>': 1, 'with</w>': 2, 'the s e</w>': 1, 'g o on i e s,</w>': 1, 'G ive</w>': 1, 'no</w>': 2, 'eff s</w>': 1, 'yeah,</w>': 2, 'w e</w>': 1, "d on ' t</w>": 1, 'g ive</w>': 1, 'eff s,</w>': 1, 'G o</w>': 1, 'fill</w>': 2, 'my</w>': 2, 'cu p</w>': 1, 'y o</w>': 1, 'g o</w>': 1, 'cup ,</w>': 1, 'Y ou </w>': 1, 'h ea r d</w>': 1, 'that</w>': 2, 'sl um s</w>': 1, 'm a d e</w>': 

**References**
1. [Speech and Language Processing - Chapter 2](https://web.stanford.edu/~jurafsky/slp3/2.pdf)
2. [Byte Pair Encoding — The Dark Horse of Modern NLP](https://towardsdatascience.com/byte-pair-encoding-the-dark-horse-of-modern-nlp-eb36c7df4f10)
